<a href="https://colab.research.google.com/github/jaimeromeroviana/Ipynbs-Colab/blob/main/Non_simulator_MEC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![texto alternativo](https://drive.google.com/uc?id=1sixQEBXqJm3-sKWbkECgt9yCkuolAd7T)



In [ ]:
import keras
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import LSTM, Dense, Flatten
import numpy as np

POSITIVE_REWARD = 1
NEGATIVE_REWARD = -1

#Neural Network
model = Sequential()
model.add(LSTM(10, return_sequences=False, input_shape=(1,2)))
model.add(Dense(10, activation = 'relu'))
model.add(Dense(2))
model.compile(loss='mean_squared_error', optimizer='adam')
model.summary()

#Target Network
target_model = Sequential()
target_model.add(LSTM(10, return_sequences=False,input_shape=(1,2)))
target_model.add(Dense(10, activation = 'relu'))
target_model.add(Dense(2))
target_model.compile(loss='mean_squared_error', optimizer='adam')
target_model.summary()
target_model.set_weights(model.get_weights())
'''
batch = np.array([3,4])
np_batch = np.reshape(batch, (1,1,batch.shape[0]))
print("batch shape",np.shape(np_batch))
print(batch)
q_list = model.predict(np_batch) 
print(q_list)
'''
# message=1-->avisa message=0-->no avisa
def step( time, message): # time and message as argument
  real_time = 13 # 13:00 
  reward = 0
  if time == real_time and message == 1:
    reward = POSITIVE_REWARD
    print("PREMIO")
  elif time == real_time and message == 0:
    reward = NEGATIVE_REWARD
    print("CASTIGO")
  elif time != real_time and message == 0:
    reward = NEGATIVE_REWARD
    print("CASTIGO")
  elif time != real_time and message == 1:
    reward = NEGATIVE_REWARD
    print("CASTIGO")
  return reward

     

Model: "sequential_51"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_49 (LSTM)               (None, 10)                520       
_________________________________________________________________
dense_88 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_89 (Dense)             (None, 2)                 22        
Total params: 652
Trainable params: 652
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_52"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_50 (LSTM)               (None, 10)                520       
_________________________________________________________________
dense_90 (Dense)             (None, 10)                110       
________________________________

In [ ]:
import numpy as np
print(np.random.randint(0,2))

0


In [ ]:
dias = 7
minutos = 1440
DISCOUNT = 0.99
x = 0
y = 0
batch_train = []
experience = []

for i in range(dias):
  day = i
  print("dia ", day)
      
  for j in range(24):
    hour=j
    """batch_predict=np.zeros((3,4))
    if(j==775):
      x= x+1
      y = 2*x
      batch_predict = np.array([[x], [y], [i], [j]])
    elif(j>775 && j<785):
      x= x+1
      y = 2*x
      batch_predict = np.insert(batch, [i], [[x],[y],[i],[j]], axis=1)"""

    print("son las ", hour)
    batch_predict = [day, hour]
    np_batch_predict = np.array(batch_predict)
    np_batch_predict = np.reshape(np_batch_predict, (1,1,batch.shape[0]))
    q_list = model.predict(np_batch_predict) 
    message = np.argmax(q_list)
    
    reward = step(message, hour)
    #newrow = [day,hour,message,reward,hour+1]
    #batch_train = np.vstack([batch_train, newrow])
    data = [day,hour]
    future_data = [day, hour+1]
    experience=[data,message,reward,future_data]
    batch_train.append(experience)

    if(hour%4==0):
      state = [element[0] for element in batch_train]
      future_state = [element[3] for element in batch_train]

      # transform list to numpy array
      np_state = np.array(state)
      np_state = np.reshape(np_state,(np_state.shape[0],1,np_state.shape[1]))
      print("npshape",np.shape(np_state))
      np_future_state = np.array(future_state)
      print(np_future_state)
      np_future_state = np.reshape(np_future_state,(np_future_state.shape[0],1,np_future_state.shape[1]))

      qs_list = model.predict(np_state)
      future_qs_list = target_model.predict(np_future_state)

      X = []
      Y = []


      for index, (current_state, message, reward, future_state) in enumerate(batch_train):

          
          max_future_q = np.max(future_qs_list[index])
          new_q = reward + DISCOUNT * max_future_q
          print("new_q %d", new_q)
          
          current_qs = qs_list[index]
          current_qs[message] = new_q

          #print("currento qs",current_qs)
          X.append(current_state)
          np_X=np.array(X)
          np_X=np.reshape(np_X,(np_X.shape[0],1,np_X.shape[1]))
          #print("x shape",np.shape(np_X))
          Y.append(current_qs)
          np_Y=np.array(Y)
          #print("y shape",np.shape(np_Y))

      model.fit(np_X/24, np_Y, batch_size=12, epochs=10, verbose=1, shuffle=True)
    """train:
    sample
    pillas un array de estados del sample
    haces un predict con esos estados, obteniendo los valores q de cada accion
    repites con el estado futuro, obteniendo los valores q de cada accion futura (esto lo sacas de la target)
    para cada sample coges el máximo q(accion tomada) y le sumas el reward
    pillas el valor q de la mejor accion futura y se lo sumas al reward con un descuento
    cambias el nuevo valor q en el array de valores q
    fit con entrada la lista de estados y salida los valores q que acabas de calcular
    
    
    Mejoras:


    -Posiciones 
    -Tiempo de levantamiento
    -Tiempo de servicio maximo levantado
    -Tiempo de mensaje
    -Usuarios diferentes
    -Recompensa variable
    -Wheather
"""



In [ ]:
import numpy as np
x = 0
y = 0
trial=[[1,1] [1,1]]
print("TRIAL",trial)
batch=np.zeros((3,4))
batch_train=([[0],[0],[0],[0],[0]])
print (batch)
for i in range(5):
  if(i==0):
    x= x+1
    y = 2*x
    batch = np.array([x, y, i, 4])
    print (batch)
  else:
    x= x+1
    y = 2*x
    newrow = [x,y,i,4]
    batch = np.vstack([batch, newrow])
    print(batch)

TypeError: ignored